# National shares of global GHG emissions
#### Christopher Callahan
#### Christopher.W.Callahan.GR@dartmouth.edu

#### Mechanics
Dependencies

In [1]:
import xarray as xr
import numpy as np
import sys
import os
import datetime
import pandas as pd
import geopandas as gp
import descartes

Data locations

In [2]:
loc_emissions = "../Data/Emissions/"
loc_shp = "../Data/CountryShapefile/"
loc_shares = "../Data/Emissions/Shares/"

Shapefile

In [3]:
shp = gp.read_file(loc_shp)
iso_shp = shp.ISO3.values

Species

In [4]:
ceds_species = ["CH4","CO2","NOx"]

Years

In [5]:
y1_in = 1750
y2_in = 2014

y1 = 1850
y2 = 2014
years = np.arange(y1,y2+1,1)

#### Analysis

Read in global carbon project territorial vs. consumption for later

In [6]:
natl_emissions_territorial = pd.read_csv(loc_emissions+"NationalEmissions_Territorial.csv",header=None)
natl_emissions_consumption = pd.read_csv(loc_emissions+"NationalEmissions_Consumption.csv",header=None)

In [7]:
iso_lookup = pd.read_csv(loc_emissions+"ISO_Lookup_for_Consumption.csv")
iso_lookup_dict = {}
for i in np.arange(0,len(iso_lookup.ISOCODE.values),1):
    isocode = iso_lookup.iloc[i,:]["ISOCODE"]
    name = iso_lookup.iloc[i,:]["NAME"]
    iso_lookup_dict[isocode] = name

Now calculate territorial and consumption-based shares of the species we're interested in

In [8]:
for s in np.arange(0,len(ceds_species),1):
    species = ceds_species[s]
    print(species)
    
    ceds_global_emissions_in = pd.read_csv(loc_emissions+"CEDS_v_2019_12_23-final_emissions/CEDS_"+species+"_global_emissions_by_fuel_v_2019_12_23.csv")
    ceds_global_emissions = ceds_global_emissions_in.loc[:,"X"+str(y1):"X"+str(y2)].sum(axis=0).values
    ceds_global_emissions_cumul = np.cumsum(ceds_global_emissions)
     
    ceds_emissions_in = pd.read_csv(loc_emissions+"CEDS_v_2019_12_23-final_emissions/CEDS_"+species+"_emissions_by_country_v_2019_12_23.csv")
    ceds_emissions = xr.DataArray(np.full((len(iso_shp),len(years)),np.nan),
                         coords=[iso_shp,years],
                         dims=["iso","time"])
    ceds_iso = ceds_emissions_in["iso"].values

    
    for ii in np.arange(0,len(iso_shp),1):
        i = iso_shp[ii]
        ystr1 = str(y1)
        ystr2 = str(y2)
        if i.lower() in ceds_iso:
            ceds_emissions.loc[i,:] = ceds_emissions_in.loc[ceds_emissions_in["iso"].values==i.lower(),"X"+ystr1:"X"+ystr2].values[0]
        else:
            ceds_emissions.loc[i,:] = np.full(len(years),np.nan)
            
    # change units if necessary
    if species == "CO2":
        scaling = (1e6*3.67)
        unit = "GtC"
    if species == "CH4":
        unit = "Mt"
        scaling = 1e3
        
    # CO2 should be in units of GtC
    # CH4 should be units of Mt (starts in kt)
    # NOx should be units of MtN -- no easy way to do this conversion so
         # we'll just calculate shares and not absolute amounts
        
    emissions_shares = xr.DataArray(np.full((len(iso_shp),len(years)),np.nan),
                                    coords=[iso_shp,years],
                                    dims=["iso","time"])
    
    for ii in np.arange(0,len(iso_shp),1):
        emissions_iso = np.where(np.isnan(ceds_emissions.loc[iso_shp[ii],:].values),0,ceds_emissions.loc[iso_shp[ii],:].values)
        emissions_shares[ii,:] = emissions_iso/ceds_global_emissions
    
    for j in np.arange(0,4,1):
        if j < 3:
            accounting = "Territorial"
        else:
            accounting = "Consumption"
        print(accounting)
        
        if j == 0:
            y1_out = 1850
            y2_out = 2014
        elif j == 1:
            y1_out = 1960
            y2_out = 2014
        else:
            y1_out = 1990
            y2_out = 2014
        yrs_out = np.arange(y1_out,y2_out+1,1) 
        
        if accounting == "Territorial":
            # no alterations to account for consumption-based emissions
            accounting_scaling = 1
            
        elif accounting == "Consumption":
            
            accounting_scaling = xr.DataArray(np.full((len(iso_shp),len(yrs_out)),np.nan),
                                              coords=[iso_shp,yrs_out],
                                              dims=["iso","time"])
            yr_indices = (natl_emissions_territorial.iloc[:,0].values>=y1_out)&(natl_emissions_territorial.iloc[:,0].values<=y2_out)
            
            for kk in np.arange(0,len(iso_shp),1):
                
                iso_code_lookup = iso_lookup_dict[iso_shp[kk]].upper()
                in_emissions_data = iso_code_lookup in natl_emissions_territorial.loc[0,:].values
                
                if iso_code_lookup in natl_emissions_territorial.loc[0,:].values:
                    territorial_str_vals = natl_emissions_territorial.loc[yr_indices,natl_emissions_territorial.loc[0,:].values==iso_code_lookup].values
                    consumption_str_vals = natl_emissions_consumption.loc[yr_indices,natl_emissions_consumption.loc[0,:].values==iso_code_lookup].values
                    territorial_vals = np.array([float(y) for y in territorial_str_vals])
                    consumption_vals = np.array([float(y) for y in consumption_str_vals])

                    accounting_scaling[kk,:] = consumption_vals/territorial_vals
                    
        emissions_shares_out = emissions_shares.loc[:,y1_out:y2_out]*accounting_scaling
        emissions_shares_out.name = "shares"
        emissions_shares_out.attrs["creation_date"] = str(datetime.datetime.now())
        emissions_shares_out.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
        emissions_shares_out.attrs["variable_description"] = "National shares of yearly fossil fuel and cement production emissions from "+species
        emissions_shares_out.attrs["created_from"] = os.getcwd()+"/FaIR_GHGEmissions_Input.ipynb"
        
        fname_out = loc_shares+"CEDS_National_"+species+"Emissions_Yearly_Shares_"+accounting+"_"+str(y1_out)+"-"+str(y2_out)+".nc"
        emissions_shares_out.to_netcdf(fname_out,mode="w")
        print(fname_out)
        
        # we also want a list of countries with nan co2, so we can exclude them in later scripts
        if species=="CO2":
            
            emissions_are_good = xr.DataArray(np.zeros(len(emissions_shares_out.iso)),coords=[iso_shp],dims=["iso_attr"])
            for i in emissions_shares_out.iso:
                em = emissions_shares_out.loc[i,:]
                emissions_are_good[list(emissions_shares_out.iso.values).index(i)] = bool(len(em) == len(em[(~np.isnan(em))&(~np.isinf(em))]))
            
            emissions_are_good.name = "valid_countries"
            emissions_are_good.attrs["creation_date"] = str(datetime.datetime.now())
            emissions_are_good.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
            emissions_are_good.attrs["variable_description"] = "Countries that should be included in later counts"
            emissions_are_good.attrs["created_from"] = os.getcwd()+"/FaIR_GHGEmissions_Input.ipynb"

            fname_out = loc_shares+"CEDS_Valid_Countries_"+accounting+"_"+str(y1_out)+"-"+str(y2_out)+".nc"
            emissions_are_good.to_netcdf(fname_out,mode="w")
            print(fname_out)

CH4
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_CH4Emissions_Yearly_Shares_Territorial_1850-2014.nc
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_CH4Emissions_Yearly_Shares_Territorial_1960-2014.nc
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_CH4Emissions_Yearly_Shares_Territorial_1990-2014.nc
Consumption


/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in true_divide
/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide


/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_CH4Emissions_Yearly_Shares_Consumption_1990-2014.nc
CO2
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_CO2Emissions_Yearly_Shares_Territorial_1850-2014.nc
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_Valid_Countries_Territorial_1850-2014.nc
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_CO2Emissions_Yearly_Shares_Territorial_1960-2014.nc
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_Valid_Countries_Territorial_1960-2014.nc
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_CO2Emissions_Yearly_Shares_Territorial_1990-2014.nc
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_Valid_Countries_Territorial_1990-2014.nc
Consu

/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in true_divide
/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide


/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_CO2Emissions_Yearly_Shares_Consumption_1990-2014.nc
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_Valid_Countries_Consumption_1990-2014.nc
NOx
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_NOxEmissions_Yearly_Shares_Territorial_1850-2014.nc
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_NOxEmissions_Yearly_Shares_Territorial_1960-2014.nc
Territorial
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_NOxEmissions_Yearly_Shares_Territorial_1990-2014.nc
Consumption


/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in true_divide
/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in true_divide


/dartfs-hpc/rc/lab/C/CMIG/ccallahan/National_Attribution/Data/Emissions/Shares/CEDS_National_NOxEmissions_Yearly_Shares_Consumption_1990-2014.nc
